In [ ]:
!pip install nemo_toolkit['all']
!pip install nemo_text_processing
!pip install pytorch-lightning
!pip install torchaudio

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 8.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is 

In [ ]:
import sys

# Definimos una clase ModelFilter ficticia para satisfacer el requisito de importación
class ModelFilter:
    pass  # Agregamos cualquier atributo o método necesario si lo requiere NeMo

# Agregamos el ModelFilter ficticio a sys.modules
sys.modules['huggingface_hub.ModelFilter'] = ModelFilter

# Ahora podemos importat Nvidia NeMo
import nemo.collections.asr as nemo_asr
import nemo.collections.tts as nemo_tts
import torchaudio
import requests

[NeMo W 2024-11-07 11:54:11 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:254: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(
    
[NeMo W 2024-11-07 11:54:11 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:265: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      def backward(ctx, grad_output):
    
[NeMo W 2024-11-07 11:54:11 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:325: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(
    
[NeMo W 2024-11-07 11:54:11 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core

In [ ]:
nemo_asr.models.EncDecCTCModel.list_available_models()

[PretrainedModelInfo(
 	pretrained_model_name=QuartzNet15x5Base-En,
 	description=QuartzNet15x5 model trained on six datasets: LibriSpeech, Mozilla Common Voice (validated clips from en_1488h_2019-12-10), WSJ, Fisher, Switchboard, and NSC Singapore English. It was trained with Apex/Amp optimization level O1 for 600 epochs. The model achieves a WER of 3.79% on LibriSpeech dev-clean, and a WER of 10.05% on dev-other. Please visit https://ngc.nvidia.com/catalog/models/nvidia:nemospeechmodels for further details.,
 	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemospeechmodels/versions/1.0.0a5/files/QuartzNet15x5Base-En.nemo
 ),
 PretrainedModelInfo(
 	pretrained_model_name=stt_en_quartznet15x5,
 	description=For details about this model, please visit https://ngc.nvidia.com/catalog/models/nvidia:nemo:stt_en_quartznet15x5,
 	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_quartznet15x5/versions/1.0.0rc1/files/stt_en_quartznet15x5.nemo
 ),
 PretrainedModelInfo(
 	pre

In [ ]:
"""
stt_en_quartznet15x5: Este es un modelo QuartzNet que ha demostrado ser muy eficiente para el reconocimiento de voz en inglés. Es una buena opción si tu tarea principal es transcribir audio en inglés.

stt_en_jasper10x5dr: Jasper es otra arquitectura de red que se ha utilizado para ASR. Este modelo es adecuado si necesitas un modelo alternativo al QuartzNet para la transcripción en inglés.

stt_es_quartznet15x5: Este modelo es específico para español. Si tu audio está en español, este es el modelo que debes utilizar."""

# Configuración para convertir Audio a Texto (ASR)
asr_model = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_en_quartznet15x5") #5

# Configuración para convertir Texto a Audio (TTS)
tts_model = nemo_tts.models.Tacotron2Model.from_pretrained(model_name="tts_en_tacotron2") #6

# Check available WaveGlow models and select an appropriate one
print(nemo_tts.models.WaveGlowModel.list_available_models()) #7
# Example: Choose the first available model
vocoder_model_name = nemo_tts.models.WaveGlowModel.list_available_models()[0]
vocoder = nemo_tts.models.WaveGlowModel.from_pretrained(model_name="tts_en_waveglow_88m")

# Función para convertir audio a texto
def audio_to_text(audio_file):
    if audio_file.startswith('http'): # Handle URLs
        response = requests.get(audio_file)
        with open('temp_audio.wav', 'wb') as f:
            f.write(response.content)
        audio_file = 'temp_audio.wav' # Use the downloaded file

    # Cargar el audio
    audio, sample_rate = torchaudio.load(audio_file)
    # Asegurarse de que el audio esté en la frecuencia correcta
    if sample_rate != 16000:
        transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        audio = transform(audio)

    # Realizar la transcripción
    # The transcribe method expects a list of file paths, not audio tensors.
    transcription = asr_model.transcribe([audio_file])
    return transcription[0]

# Función para convertir texto a audio
def text_to_audio(text, output_file="output.wav"):
    # Generar mel-spectrograma a partir del texto
    parsed = tts_model.parse(text)
    spectrogram = tts_model.generate_spectrogram(tokens=parsed)

    # Convertir mel-spectrograma a onda sonora
    audio = vocoder.convert_spectrogram_to_audio(spec=spectrogram)

    # Guardar el archivo de audio
    torchaudio.save(output_file, audio.to('cpu'), 22050)
    print(f"Audio saved to {output_file}")

from IPython.display import Audio

# Ejemplo de uso
audio_file = 'https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav'
transcription = audio_to_text(audio_file)
print("Transcription:", transcription)

text = "This is a test of the text to speech conversion."
text_to_audio(text, output_file="output.wav")

# Reproduce el audio generado en Colab
Audio("output.wav", autoplay=True)

[NeMo I 2024-11-07 12:03:47 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_1.23.0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.
[NeMo I 2024-11-07 12:03:47 cloud:64] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.23.0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo
[NeMo I 2024-11-07 12:03:47 common:924] Instantiating model from pre-trained checkpoint


[NeMo W 2024-11-07 12:03:48 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/voices/train_1k.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: /asr_set_1.2/train/train_{0..1023}.tar
    num_workers: 20
    
[NeMo W 2024-11-07 12:03:48 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
   

[NeMo I 2024-11-07 12:03:48 features:289] PADDING: 16
[NeMo I 2024-11-07 12:03:48 save_restore_connector:249] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.23.0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.
[NeMo I 2024-11-07 12:03:48 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_1.23.0/tts_en_tacotron2/319a832e78fac50399d90aee0f083927/tts_en_tacotron2.nemo.
[NeMo I 2024-11-07 12:03:48 cloud:64] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.23.0/tts_en_tacotron2/319a832e78fac50399d90aee0f083927/tts_en_tacotron2.nemo
[NeMo I 2024-11-07 12:03:48 common:924] Instantiating model from pre-trained checkpoint


 NeMo-text-processing :: INFO     :: Creating ClassifyFst grammars.
INFO:NeMo-text-processing:Creating ClassifyFst grammars.
[NeMo W 2024-11-07 12:04:27 en_us_arpabet:66] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2024-11-07 12:04:27 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /raid/datasets/tts_data/train_manifest.json
      sample_rate: 22050
      sup_data_path: null
      sup_data_types: null
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowf

[NeMo I 2024-11-07 12:04:27 features:289] PADDING: 16
[NeMo I 2024-11-07 12:04:27 features:297] STFT using exact pad
[NeMo I 2024-11-07 12:04:28 save_restore_connector:249] Model Tacotron2Model was successfully restored from /root/.cache/torch/NeMo/NeMo_1.23.0/tts_en_tacotron2/319a832e78fac50399d90aee0f083927/tts_en_tacotron2.nemo.
[PretrainedModelInfo(
	pretrained_model_name=tts_en_waveglow_88m,
	description=This model is trained on LJSpeech sampled at 22050Hz, and has been tested on generating female English voices with an American accent and Mandarin voices.,
	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemo/tts_waveglow_88m/versions/1.0.0/files/tts_waveglow.nemo,
	class_=<class 'nemo.collections.tts.models.waveglow.WaveGlowModel'>
)]
[NeMo I 2024-11-07 12:04:28 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_1.23.0/tts_waveglow/be8c69c04450b3da0ce5a38a8ef4014f/tts_waveglow.nemo.
[NeMo I 2024-11-07 12:04:28 cloud:64] Re-using file from: /root/.cache/torch/NeMo/

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

[NeMo W 2024-11-07 12:04:33 tacotron2:144] parse() is meant to be called in eval mode.


Transcription: well i don't wish to see it any more observed phoebe turning away her eyes it is certainly very likt the old portrait


[NeMo W 2024-11-07 12:04:36 tacotron2:338] Reached max decoder steps 1000.


Audio saved to output.wav
